In [23]:
import os 

In [24]:
from RAGPipeLine import Ragpipeline

pipeline = Ragpipeline()


[초기화] vector_store 초기화 완료
[초기화] retriever 초기화 완료


In [26]:
from django.db import models
import uuid # UUID(Universally Unique Identifier) 모듈을 임포트합니다. UUID는 고유한 식별자를 생성하는 데 사용됩니다.


In [28]:
class RagDocument():
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, editable=False)                 # id 필드는 UUID 형식의 기본 키(primary key)입니다. 기본값은 uuid.uuid4 함수를 사용해 생성됩니다. 이 필드는 수정할 수 없습니다(editable=False).
    file = models.FileField(upload_to='documents/', default='documents/default.txt')            # file 필드는 파일을 저장하는 필드입니다. 파일은 documents/ 디렉토리에 업로드됩니다. 기본값은 documents/default.txt 파일입니다.
    uploaded_at = models.DateTimeField(auto_now_add=True)                       # uploaded_at 필드는 파일이 업로드된 시간을 저장하는 날짜/시간 필드입니다. 

    def __str__(self):                  # __str__ 메서드는 file 필드의 파일 이름을 반환합니다. 이 메서드는 Django 관리자(admin) 인터페이스 등에서 객체를 사람이 읽을 수 있는 형태로 표시하는 데 사용됩니다.
        return self.file.name

In [29]:
obj = RagDocument

In [30]:
obj.file.path = './documents/sample.txt'

In [31]:
file_path = obj.file.path 

In [32]:
file_path

'./documents/sample.txt'

In [33]:
obj.file

<django.db.models.fields.files.FileField>

In [ ]:
# 벡터 DB에 문서 업데이트 시도
file_path = obj.file.path
with open(file_path, 'rb') as f:
    if pipeline.update_vector_db(f, obj.file):
        print("[문서업로드] 벡터 스토어 업데이트에 성공하였습니다.")
    else:
        # 유사한 문서가 존재하여 업데이트에 실패 시 저장된 문서 삭제
        obj.delete()
        os.remove(file_path)
        print("[문서업로드] 유사한 문서가 발견되어 벡터스토어 업데이트가 거절되었습니다.")


In [35]:
import pandas as pd 

In [40]:
df = pd.read_excel('./documents/크롤링_리스트.xlsx')

In [42]:
df.to_csv('./documents/크롤링_리스트.csv')

In [54]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [44]:
loader = CSVLoader(file_path="documents/크롤링_리스트.csv")
data = loader.load()

In [47]:
# 행의 수 출력
len(data)

61

In [46]:
data[:5]

[Document(metadata={'source': 'documents/크롤링_리스트.csv', 'row': 0}, page_content=': 0\n정보 출처: 인도 특허청\n세부 구분: 법률_규범\n추출할 정보 설명: \n담당자: 문승현\n우선순위: 1.0\nURL: https://iprsearch.ipindia.gov.in/PublicSearch/PublicationSearch/PatentSearchResult'),
 Document(metadata={'source': 'documents/크롤링_리스트.csv', 'row': 1}, page_content=': 1\n정보 출처: 제약산업정보포털\n세부 구분: 정책\n추출할 정보 설명: 인도 통관 및 운송에 대한 내용\n담당자: \n우선순위: 4.0\nURL: https://www.khidi.or.kr/board/view?pageNum=6&rowCnt=10&no1=50&linkId=47399284&menuId=MENU01864&maxIndex=&minIndex=&schType=0&schText=&schStartDate=&schEndDate=&boardStyle=&categoryId=&continent=&country='),
 Document(metadata={'source': 'documents/크롤링_리스트.csv', 'row': 2}, page_content=': 2\n정보 출처: 인도 무역청\n세부 구분: 법률_규범\n추출할 정보 설명: 용석 : 전체 사이트 모든 내용(셀레니움?)(크롤링시 상의 필요)\n담당자: \n우선순위: 2.0\nURL: https://www.indiantradeportal.in/index.jsp?lang=0'),
 Document(metadata={'source': 'documents/크롤링_리스트.csv', 'row': 3}, page_content=': 3\n정보 출처: 인도 무역구제총국\n세부 구분: 법률_규범\n추출할 정보 설명: 용석 : 전체 사이트 모든 내용(셀레니

In [51]:
data[0].metadata

{'source': 'documents/크롤링_리스트.csv', 'row': 0}

In [52]:
data[0].page_content

': 0\n정보 출처: 인도 특허청\n세부 구분: 법률_규범\n추출할 정보 설명: \n담당자: 문승현\n우선순위: 1.0\nURL: https://iprsearch.ipindia.gov.in/PublicSearch/PublicationSearch/PatentSearchResult'

In [58]:


loader = PyPDFLoader("documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf")
docs = loader.load()

In [59]:
docs

[Document(metadata={'source': 'documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'page': 0}, page_content='5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물\n품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도\n관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되\n며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를\n완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage\nCharge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. \n \n2) 임시 통관 \n \n인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해서는 해\n당 품목의 관세를 납부해야하는데, 다시 반출할 시 장치했던 시간에 비례하여 관세를 환급받을 수 있다. 인도 내로 반입되지 않고 바로 반\n출할 경우 최대 98%까지 환급이 가능하나, 인도 내 해당 물품이 사용될 경우 기간과 상관없이 관세의 환급이 불가하다. \n \n  ㅇ ATA Carnet \n    - ATA 까르네(ATA Carnet)는 Admission Temporaire이라는 임시통관을 뜻하는 불어와 Temporary Admission의 영어 그리고 수첩\n을 

In [62]:
docs[0].metadata

{'source': 'documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'page': 0}

In [63]:
docs[0].page_content

'5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물\n품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도\n관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되\n며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를\n완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage\nCharge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. \n \n2) 임시 통관 \n \n인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해서는 해\n당 품목의 관세를 납부해야하는데, 다시 반출할 시 장치했던 시간에 비례하여 관세를 환급받을 수 있다. 인도 내로 반입되지 않고 바로 반\n출할 경우 최대 98%까지 환급이 가능하나, 인도 내 해당 물품이 사용될 경우 기간과 상관없이 관세의 환급이 불가하다. \n \n  ㅇ ATA Carnet \n    - ATA 까르네(ATA Carnet)는 Admission Temporaire이라는 임시통관을 뜻하는 불어와 Temporary Admission의 영어 그리고 수첩\n을 뜻하는 불어 Carnet의 머리글자 조합으로, 임시 통관을 위한 수첩을 말한다. ATA 협약에는 인도를 포함하여 전 세계 71개국이 서명하\n여 발효 중이며, ATA 통관 수첩은 국제 관세 서류(

In [64]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf")
docs = loader.load()

In [65]:
docs

[Document(metadata={'source': 'documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'file_path': 'documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iText 2.1.7 by 1T3XT', 'creationDate': "D:20190305111302+09'00'", 'modDate': "D:20190305111302+09'00'", 'trapped': ''}, page_content='5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물\n품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도\n관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되\n며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를\n완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage\nCharge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. \n \n2) 임시 통관 \n \n인도에 들여온 품목을 사용하지 않고 24개월 이내에 다

In [66]:
docs[0].metadata

{'source': 'documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf',
 'file_path': 'documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf',
 'page': 0,
 'total_pages': 4,
 'format': 'PDF 1.4',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'iText 2.1.7 by 1T3XT',
 'creationDate': "D:20190305111302+09'00'",
 'modDate': "D:20190305111302+09'00'",
 'trapped': ''}

In [67]:
docs[0].page_content

'5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물\n품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도\n관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되\n며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를\n완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage\nCharge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. \n \n2) 임시 통관 \n \n인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해서는 해\n당 품목의 관세를 납부해야하는데, 다시 반출할 시 장치했던 시간에 비례하여 관세를 환급받을 수 있다. 인도 내로 반입되지 않고 바로 반\n출할 경우 최대 98%까지 환급이 가능하나, 인도 내 해당 물품이 사용될 경우 기간과 상관없이 관세의 환급이 불가하다. \n \n  ㅇ ATA Carnet \n    - ATA 까르네(ATA Carnet)는 Admission Temporaire이라는 임시통관을 뜻하는 불어와 Temporary Admission의 영어 그리고 수첩\n을 뜻하는 불어 Carnet의 머리글자 조합으로, 임시 통관을 위한 수첩을 말한다. ATA 협약에는 인도를 포함하여 전 세계 71개국이 서명하\n여 발효 중이며, ATA 통관 수첩은 국제 관세 서류(